In [1]:
from DatasetHandler.BiwiBrowser import *
biwi = readBIWIDataset(subjectList = [s for s in range(1, 2)])#

/home/mcicek/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Frames from /home/mcicek/Datasets/HeadPoses/biwi/hpdb/ have been started to read by 2018-11-26 03:50:07.972614
Subject 01's frames have been started to read 2018-11-26 03:50:07.973795
Subject 01's all frames have been read by 2018-11-26 03:50:07.973826
/home/mcicek/Datasets/HeadPoses/biwi/db_annotations.tgz has been started to read by 2018-11-26 03:50:07.973840
1 annotations have been read by 2018-11-26 03:50:08.796117


In [2]:
frames, labelsList = [], []
for inputMatrix, labels in biwi:
    frames.append(inputMatrix)
    labelsList.append(labels)

In [3]:
inputMatrix.shape

(498, 480, 640, 3)

In [4]:
inputMatrix, labels = frames[0], labelsList[0]

In [5]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import keras

In [ ]:
# Inspect the model
vgg16.summary()

In [33]:
# Verify things look as expected
full_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [6]:
patch_size = BIWI_Frame_Shape #(48, 64, 3)#

In [7]:
from keras.layers import *
from keras import Model 

In [21]:
keras.backend.clear_session()
def getFinalModel():
    dense_layer_1 = 1#int((patch_size[0] * patch_size[1]) / 1)0010#00000
    dense_layer_2 = 8
    num_classes = 6
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape= patch_size)
    block5_pool = vgg16.get_layer('block5_pool').output
    gate = (int(d) for d in block5_pool.shape[1:])
    #vgg16.summary()
    # This shape has to match the last layer in VGG16 (without top)
    dense_input  = Input(shape=gate) #patch_size) #(15, 20, 512)
    dense_output = Flatten(name='flatten')(dense_input)
    dense_output = Dense(dense_layer_1, activation='relu', name='fc1')(dense_output)
    dense_output = Dense(dense_layer_2, activation='relu', name='fc2')(dense_output)
    dense_output = Dense(num_classes, activation='softmax', name='predictions')(dense_output)

    top_model = Model(inputs=dense_input, outputs=dense_output, name='top_model')
    # Now combine the two models
    full_output = top_model(block5_pool)
    full_model  = Model(inputs=vgg16.input, outputs=full_output)

    for layer in full_model.layers[:15]:
        layer.trainable = False
    full_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return full_model

In [22]:
full_model = getFinalModel()

In [25]:
full_model.fit(inputMatrix, labels, batch_size=10, epochs=1, verbose=2)

Epoch 1/1
 - 27s - loss: 132912.7791 - acc: 1.0000
